In [1]:
import gurobipy as gp
import numpy as np

In [2]:
"""
# P_ABC_dict = {(0,0,0): 0,
#          (0,0,1): 1/3,
#          (0,1,0): 1/3,
#          (1,0,0): 1/3,
#          (0,1,1): 0,
#          (1,0,1): 0,
#          (1,1,0): 0,
#          (1,1,1): 0}

P_ABC_dict = {(0,0,0): 1/2,
         (0,0,1): 0,
         (0,1,0): 0,
         (1,0,0): 0,
         (0,1,1): 0,
         (1,0,1): 0,
         (1,1,0): 0,
         (1,1,1): 1/2}

# P_ABC dict to matrix
P_ABC = np.zeros((2,2,2))
for a,b,c in P_ABC_dict:
    P_ABC[a,b,c] = P_ABC_dict[a,b,c]
"""





v=3/4
P_ABC = np.zeros((2,2,2))
for a,b,c in np.ndindex(2,2,2):
    P_ABC[a,b,c] = (1-v)/7
P_ABC[1,1,1] = v

P_A = np.array([sum([P_ABC[a,b,c] for b,c in np.ndindex(2,2)]) for a in range(2)])
P_B = np.array([sum([P_ABC[a,b,c] for a,c in np.ndindex(2,2)]) for b in range(2)])
P_C = np.array([sum([P_ABC[a,b,c] for a,b in np.ndindex(2,2)]) for c in range(2)])

In [3]:
m = gp.Model()
Q_Ab_BC_giv_As = m.addMVar((2, 2, 2, 2), name="Q", vtype=gp.GRB.CONTINUOUS, lb=0, ub=1) # A interupted
R_A1A2_B1B2_C1C2_giv_As = m.addMVar((2, 2, 2, 2, 2, 2, 2), name="R", vtype=gp.GRB.CONTINUOUS, lb=0, ub=1) # inflation
m.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-09


In [4]:
# consistency
for a, b, c in np.ndindex(2,2,2):
    m.addConstr(P_ABC[a,b,c] == Q_Ab_BC_giv_As[a,b,c,a])


## normalization
for a_s in np.ndindex(2):
    m.addConstr(R_A1A2_B1B2_C1C2_giv_As[:,:,:,:,:,:,a_s].sum() == 1)
    m.addConstr(Q_Ab_BC_giv_As[:,:,:,a_s].sum() == 1)


"""
## marginalization, should be trivial
for var, a_s in np.ndindex(2,2):
    m.addConstr(Q_Ab_BC_giv_As[var,:,:,a_s].sum() == P_A[var]) # Q(Ab|As) = P(A)
    m.addConstr(Q_Ab_BC_giv_As[:,:,var,a_s].sum() == P_C[var]) # Q(C|As) = P(C)
"""

## do conditional on P is a normal conditional on Q
def P_B_do_A(b,a):
    return Q_Ab_BC_giv_As[:,b,:,a].sum()

In [5]:
## graph to graph relations:

# R(A1,B1,C1|A#) = Q(Ab,B,C|A#)
for a,b,c,a_s in np.ndindex(2,2,2,2):
    m.addConstr(R_A1A2_B1B2_C1C2_giv_As[a,:,b,:,c,:,a_s].sum() == Q_Ab_BC_giv_As[a,b,c,a_s])

# R(A2,B2,C2|A#) = P(A)P(C)Q(B|A#) = Q(A)Q(C)Q(B|A#)
for a,b,c,a_s in np.ndindex(2,2,2,2):
    m.addConstr(R_A1A2_B1B2_C1C2_giv_As[:,a,:,b,:,c,a_s].sum() == P_A[a]*P_C[c]*Q_Ab_BC_giv_As[:,b,:,a_s].sum())
    # also == Q_Ab_BC_giv_As[a,b,c,a_s]?

# R(A1,B2|A#) = Q(Ab,B|A#)
for a_b, a_s, b in np.ndindex(2,2,2):
    m.addConstr(R_A1A2_B1B2_C1C2_giv_As[a_b,:,:,b,:,:,a_s].sum() == Q_Ab_BC_giv_As[a_b,b,:,a_s].sum())

# # R(C2,A2|A#) = P(A)P(C)
# for c, a, a_s in np.ndindex(2,2,2):
#     m.addConstr(R_A1A2_B1B2_C1C2_giv_As[:,a,:,:,:,c,a_s].sum() == P_A[a]*P_C[c])

# R(A1,A2,C1,C2|A# = 0) = R(A1,A2,C1,C2|A# = 1)
for a1,a2,c1,c2 in np.ndindex(2,2,2,2):
    m.addConstr(R_A1A2_B1B2_C1C2_giv_As[a1,a2,:,:,c1,c2,0].sum() == R_A1A2_B1B2_C1C2_giv_As[a1,a2,:,:,c1,c2,1].sum())

In [6]:
# independence on Q

# (Ab ⫫ As| B) on Q: Q(Ab|B,A#=0) = Q(Ab|B, A#=1)
for a_b in range(2):
    m.addConstr(Q_Ab_BC_giv_As[a_b,:,:,0].sum() == Q_Ab_BC_giv_As[a_b,:,:,1].sum())

# (C ⫫ As| B) on Q: Q(C|B,A#=0) = Q(C|B,A#=1)
for c in range(2):
    m.addConstr(Q_Ab_BC_giv_As[:,:,c,0].sum() == Q_Ab_BC_giv_As[:,:,c,1].sum())

objective is to minimize $|P(B=1|do(A=0)) - P(B=1|do(A=1))|$  
can do in linear programming by:  

minimizing $t$ s.T: $$P(B=1|do(A=0)) - P(B=1|do(A=1)) \le t \\ P(B=1|do(A=1)) - P(B=1|do(A=0))\le t$$

In [7]:
t = m.addVar(name="t", vtype=gp.GRB.CONTINUOUS, lb=0, ub=10)
a1, a2 = 0,1
b = 1

m.addConstr(P_B_do_A(b,a1) - P_B_do_A(b,a2) <= t)
m.addConstr(P_B_do_A(b, a2) - P_B_do_A(b, a1) <= t)
m.update()

In [8]:
m.setObjective(t, gp.GRB.MINIMIZE)
m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 74 rows, 145 columns and 810 nonzeros
Model fingerprint: 0x6a65e442
Coefficient statistics:
  Matrix range     [2e-02, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+01]
  RHS range        [4e-02, 1e+00]
Presolve removed 30 rows and 16 columns
Presolve time: 0.00s
Presolved: 44 rows, 129 columns, 399 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   7.144877e+00   0.000000e+00      0s
      61    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 61 iterations and 0.01 seconds (0.00 work units)
Optimal objective  0.000000000e+00


In [9]:
if m.status == gp.GRB.OPTIMAL:
    print(f"t={t.X}\nP_B_do_A({b},{a1})={Q_Ab_BC_giv_As[:,b,:,a1].X.sum()}\nP_B_do_A({b},{a2})={Q_Ab_BC_giv_As[:,b,:,a2].X.sum()}")
    print(f"|P_B_do_A({b},{a1}) - P_B_do_A({b},{a2})| =",abs(Q_Ab_BC_giv_As[:,b,:,a1].X.sum() - Q_Ab_BC_giv_As[:,b,:,a2].X.sum()))
else:
    print("No solution found")

t=0.0
P_B_do_A(1,0)=0.7857142857142858
P_B_do_A(1,1)=0.7857142857142857
|P_B_do_A(1,0) - P_B_do_A(1,1)| = 1.1102230246251565e-16
